## Setup
Installs and imports necessary modules from Github.

Trouble running this? Try restarting the Colab session ("Runtime" --> "Restart Session").

If you're working in your own branch in the Github repo, change the wget commands to point towards that branch.

In [11]:
!mkdir -p modules
!wget -q https://raw.githubusercontent.com/cathat00/NMA_B-T_Project/main/modules/network.py -O modules/network.py
!wget -q https://raw.githubusercontent.com/cathat00/NMA_B-T_Project/main/modules/tasks.py -O modules/tasks.py
!wget -q https://raw.githubusercontent.com/cathat00/NMA_B-T_Project/main/modules/bci.py -O modules/bci.py
!wget -q https://raw.githubusercontent.com/cathat00/NMA_B-T_Project/main/modules/configs.py -O modules/configs.py
!wget -q https://raw.githubusercontent.com/cathat00/NMA_B-T_Project/main/modules/plots.py -O modules/plots.py

import numpy as np
from os.path import join
import modules.configs as configs
import modules.plots as plots

##Train and Evaluate the Model

In [10]:
# ====================
# == Initialization ==
# ====================

experiments = [1,2,3,4,5,6,8,9,10,11,12]
all_pca_figs = []
num_targets_list = []
eigenval_list = []

# =====================
# == Run Experiments ==
# =====================

for ntargets in experiments:

  # ======================
  # == Setup Experiment ==
  # ======================

  exp_name = f"{ntargets}Targets" # Name of this experiment
  exp_out_dir = join(exp_out_dir, )

  cfg = configs.BasicExperimentConfig(ntargets=ntargets)
  task = cfg.task # The task the RNN will learn
  rnn = cfg.rnn # Recurrent Neural Network
  bci = cfg.bci # Brain computer interface
  feedback = cfg.feedback # Mathematically optimal feedback

  # =======================
  # == Simulate Learning ==
  # =======================

  # -- Train the RNN
  print(f"Training the RNN for {cfg.ntrials} trials...")
  rnn.relearn(
      ntrials=cfg.ntrials,
      ext=task.stimuli,
      ntstart=task.stim_length,
      decoder=bci.decoder,
      feedback=feedback,
      target=task.targets,
  )

  # -- Compute Manifold
  print(f"Computing manifold over {cfg.ntrials_manifold} trials...")
  np.random.seed(2) # Set seed for manifold calculation...
  manifold_data = rnn.get_manifold(task.stimuli, task.stim_length, ntrials=cfg.ntrials_manifold)
  manifold_activity = manifold_data['proj_reshaped']
  eigenvalues = manifold_data['eigenvals']

  # ===========================================
  # == Visualize Results For This Experiment ==
  # ===========================================
  pca_fig = plots.plot_pca_summary(manifold_activity, eigenvalues)

  # ======================================
  # == Save Results For This Experiment ==
  # ======================================
  cfg.save("result.pkl", manifold_data)

Training the RNN for 80 trials...


  2%|▎         | 2/80 [00:13<08:44,  6.73s/it]


KeyboardInterrupt: 